# Import Data
In order to train our model, we are going to import time series data of room temperature.

With our RNN, we are going to try to forecast the temperature of a room given a certain air flow temperature.

You can find this dataset here: https://www.kaggle.com/datasets/vitthalmadane/ts-temp-1/data?select=MLTempDataset.csv

In [1]:
import kagglehub

# Download latest version of dataset to local filesystem
path = kagglehub.dataset_download("vitthalmadane/ts-temp-1")

print("Path to dataset files:", path)

Path to dataset files: /home/noah/.cache/kagglehub/datasets/vitthalmadane/ts-temp-1/versions/2


## Read Data into Memory
For this workshop, we are going to use pandas to read the data into memory and convert it into a numpy array to train our model

In [2]:
import pandas as pd
import os
import numpy as np

file_path = os.path.join(path, 'MLTempDataset.csv')
temperature_data = pd.read_csv(file_path)

In [3]:
temperature_data.info()

<class 'pandas.DataFrame'>
RangeIndex: 6676 entries, 0 to 6675
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  6676 non-null   int64  
 1   Datetime1   6676 non-null   int64  
 2   DAYTON_MW   6676 non-null   float64
 3   Datetime    6676 non-null   str    
dtypes: float64(1), int64(2), str(1)
memory usage: 208.8 KB


## Data Visualization

In order to train our model, we need to understand what we are training the model on.

If we look at the top 5 rows of our temperature data, we are given 4 columns:

1. Unnamed: This represents the index of the data, since our data is in order, we can ignore this.
2. Datetime1: This is the hour of the day between 0 and 23
3. DAYTON_MW: This is the temperature of the room in Celsius
4. Datetime: This is the Date and hour of the day in a Datetime Format

We want our model to forecast temperature over time. So, we will treat our temperature as our label and we will Datetime as the time step.

In [4]:
temperature_data.head()

,Unnamed: 0,Datetime1,DAYTON_MW,Datetime
0,0,0,20.867,2022-01-04 00:00:00
1,1,1,21.000,2022-01-04 01:00:00
2,2,2,20.867,2022-01-04 02:00:00
3,3,3,20.650,2022-01-04 03:00:00
4,4,4,20.400,2022-01-04 04:00:00


# Data Preprocessing
Continuing with the data aspect of machine learning, we need to preprocess our data in a way that facilitates training for time series data.

As part of this, we will need to standardize our data.

In [5]:
# Convert pandas data to numpy for training
temperature_data = temperature_data.to_numpy()
temperature_data

array([[0, 0, 20.867, '2022-01-04 00:00:00'],
       [1, 1, 21.0, '2022-01-04 01:00:00'],
       [2, 2, 20.867, '2022-01-04 02:00:00'],
       ...,
       [6673, 21, 26.45, '2022-10-09 01:00:00'],
       [6674, 22, 25.9, '2022-10-09 02:00:00'],
       [6675, 23, 25.567, '2022-10-09 03:00:00']],
      shape=(6676, 4), dtype=object)

In [6]:
# Create function to create dataset
# Basically, we are saying, take 60 sequential temperatures, then, given this information
# predict what the next temperature would be.
def create_dataset(data, time_step=60):
    X, y = [], []
    for i in range(len(data) - time_step - 1):
        X.append(data[i:(i + time_step), 2])
        y.append(data[i + time_step, 2])
    return np.array(X).astype('float64'), np.array(y).astype('float64')

# X, y = create_dataset(temperature_data)
# X = X.reshape(X.shape[0], X.shape[1], 1)


In [7]:
X, y = create_dataset(temperature_data)

In [8]:
X = X.reshape(X.shape[0], X.shape[1], 1)

In [9]:
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Build the Model

Using Tensorflow and Keras, we are going to build a Recurrent Nueral Network to forecast room temperature. In particular, we are going to use a Gated Recurrent Unit Model. 

At a high level, this addresses the vanishing gradient problem, allowing them to reliably predict over longer periods of time.

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, GRU
model = Sequential([
    # GRU(64, activation='tanh', return_sequences=True, input_shape=(10, 5)),  # First GRU layer
    GRU(64, activation='tanh'),  # Second GRU layer
    Dense(1)  # Output layer for binary classification
])

model.compile(optimizer='adam', loss='mse', metrics=['r2_score'])
model.summary()

2026-01-26 22:37:22.623044: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-26 22:37:22.650644: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-26 22:37:23.204242: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
I0000 00:00:1769485043.656707   76119 gpu_device.cc:2020] Created device /job:localhost/rep

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru (GRU)                       │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

## Train the model
Now, we are going to train our model to fit our data and hopefully get a descent temperature forecaster

In [11]:
model.fit(X_train, y_train, epochs=20, batch_size=64)

predictions = model.predict(X_test)

Epoch 1/20


2026-01-26 22:37:24.383998: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91002


83/83 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 280.0976 - r2_score: -4.3851
Epoch 2/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 101.0410 - r2_score: -0.9426
Epoch 3/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 57.6578 - r2_score: -0.1085
Epoch 4/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 33.7979 - r2_score: 0.3502
Epoch 5/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 22.3633 - r2_score: 0.5700
Epoch 6/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 16.6705 - r2_score: 0.6795
Epoch 7/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 13.6076 - r2_score: 0.7384
Epoch 8/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 11.9246 - r2_score: 0.7707
Epoch 9/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 10.8340 - r2_score: 0.7917
Epoch 10/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 10.1604 - r2_score: 0.8047
Epoch 11/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 9.6791 - r2_score: 0.8139
Epoch 12/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 9.3132 - r2

In [12]:
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)

print('\nTest accuracy:', test_acc)

42/42 - 0s - 8ms/step - loss: 3.0048 - r2_score: 0.8580

Test accuracy: 0.8579612970352173


In [13]:

model.predict(X_train, verbose=0)[:10]

array([[24.038025],
       [24.225903],
       [24.945562],
       [25.326895],
       [25.249283],
       [12.122084],
       [10.242821],
       [10.996746],
       [17.140553],
       [12.288088]], dtype=float32)

In [14]:
y_train[:10].transpose()

array([24.867, 25.5  , 26.   , 26.267,  9.633,  9.267,  8.85 , 15.6  ,
       12.267,  9.067])

## Exporting the model
At this point, we have a satisfactory model. At this point, we are going to export this model to run.

In [15]:
!mkdir models
model.save('models/my_model.h5')

mkdir: cannot create directory ‘models’: File exists


## Optimization
There are techniques that we can use to reduce the size of Nueral Networks to reduce their size
and their computational cost.

Some of these include:
1. Weight Pruning
2. Quantization
3. Weight Clustering

## Quantization
Quantization aware training emulates inference-time quantization, creating a model that downstream tools will use to produce actually quantized models. The quantized models use lower-precision (e.g. 8-bit instead of 32-bit float), leading to benefits during deployment.

### Deploy with quantization

Quantization brings improvements via model compression and latency reduction. With the API defaults, the model size shrinks by 4x, and we typically see between 1.5 - 4x improvements in CPU latency in the tested backends. Eventually, latency improvements can be seen on compatible machine learning accelerators, such as the EdgeTPU and NNAPI.